Examples Of Pyspark ML

In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Missing').getOrCreate()

In [2]:
## Read The dataset
training = spark.read.csv('test1.csv',header=True,inferSchema=True)

In [3]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [4]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [5]:
training.columns

['Name', 'age', 'Experience', 'Salary']

[Age,Experience]----> new feature--->independent feature

In [6]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")
#A feature transformer that merges multiple columns into a vector column.

In [19]:
output=featureassembler.transform(training)#there is no such featureassembler.fit(training) attribute in VectorAsse

In [8]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [9]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [10]:
finalized_data=output.select("Independent Features","Salary")

In [11]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [12]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')#featuresCol represents independent features and labelCol represents dependent feature
regressor=regressor.fit(train_data)

In [13]:
### Coefficients
regressor.coefficients

DenseVector([-383.9733, 1711.1853])

In [14]:
### Intercepts
regressor.intercept

23998.330550919385

In [15]:
### Prediction
pred_results=regressor.evaluate(test_data)

In [16]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000| 17646.07679465793|
|          [23.0,2.0]| 18000|18589.315525876566|
+--------------------+------+------------------+



In [17]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError,pred_results.rootMeanSquaredError

(1617.6961602672473, 3674507.5961331767, 1916.9005180585602)

Estimators:

What they do: An Estimator must learn something from your data. It is fit on a DataFrame to produce a model (which is a Transformer).

Analogy: A student learning from a textbook. The learning process is fit(). Once learned, the student becomes a Transformer and can transform (answer questions on) new, similar problems.

Examples: Imputer, StringIndexer, StandardScaler

Why .fit() is needed: The Imputer needs to calculate the mean, median, or mode from the specific data you provide. This calculated value is stored in the resulting ImputerModel (the Transformer).

Transformers:

What they do: A Transformer does not need to learn from the data. It simply applies a pre-defined transformation rule or function to a DataFrame.

Analogy: A simple function, like adding 1 to every number in a list. The rule is fixed and doesn't change based on the data.

Examples: VectorAssembler, Tokenzier, StopWordsRemover

Why .fit() is NOT needed: The VectorAssembler's rule is simple and pre-defined: "take the specified columns and combine their values into a single vector." This rule doesn't depend on the data's content (like its mean or unique values), so it doesn't need a training phase